In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [3]:
!pip install evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [4]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
import json

# Đọc dữ liệu từ file JSON
with open('/content/drive/MyDrive/question.json', 'r', encoding='utf-8') as f:
    data_list = json.load(f)["data"]

# Tạo cấu trúc dữ liệu SQuAD
squad_data = {
    "version": "1.1",  # Hoặc phiên bản SQuAD mà bạn đang sử dụng
    "data": []
}

# Xây dựng cấu trúc SQuAD
for item in data_list:
    # Tìm phần tử có tiêu đề tương ứng hoặc tạo mới
    article = next((art for art in squad_data["data"] if art["title"] == item["title"]), None)

    if article is None:
        article = {
            "title": item["title"],
            "paragraphs": []
        }
        squad_data["data"].append(article)

    # Thêm các đoạn văn bản và danh sách câu hỏi
    for paragraph in item["paragraphs"]:
        for qa in paragraph["qas"]:
            paragraph_data = {
                "context": paragraph["context"],
                "qas": [
                    {
                        "id": qa["id"],
                        "question": qa["question"],
                        "answers": [
                            {
                                "text": answer["text"],
                                "answer_start": answer["answer_start"]
                            } for answer in qa.get("answers", [])
                        ],
                        "is_impossible": qa.get("is_impossible", False)
                    }
                ]
            }

            # Thêm đoạn văn bản vào bài viết
            article["paragraphs"].append(paragraph_data)

# Ghi ra file
output_path = '/content/drive/MyDrive/BART/data/Thaismart.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(squad_data, f, ensure_ascii=False, indent=2)

print(f'Data has been converted and saved to {output_path}')


Data has been converted and saved to /content/drive/MyDrive/BART/data/Thaismart.json


In [6]:
from datasets import load_dataset
raw_datasets = load_dataset("/content/drive/MyDrive/BART/utils/Bản sao của viquad (1) (2).py")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:929: FutureWarning: The repository for Bản sao của viquad (1) (2) contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /content/drive/MyDrive/BART/utils/Bản sao của viquad (1) (2).py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [7]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 892
    })
})


In [8]:

print(raw_datasets['train'][-1]['id'])

101-07


In [9]:
print("ID: ", raw_datasets["train"][0]["id"])
print("Title: ", raw_datasets["train"][0]["title"])
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

ID:  01-01
Title:  Thạc sĩ Lê Tấn Hùng
Context:  Thạc sĩ Lê Tấn Hùng hiện đang làm việc tại bộ môn Công nghệ Phần mềm từ năm 2000. ThS Hùng có các hướng nghiên cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos. Email của ông là: hunglt@soict.hust.edu.vn và có trang web cá nhân tại: http://www.soict.hust.edu.vn/~hunglt. Ông đã tốt nghiệp Thạc sĩ tại Đại học Châu Á, Thái Lan năm 1998 và Cử nhân tại Đại học Tổng hợp Sofia năm 1994. ThS. Hùng giảng dạy các môn học bao gồm Đồ họa máy tính, Đa phương tiện và Hệ thống giải trí, Đồ Họa và Hiện thực ảo, Đạo đức Máy tính, và Sáng Tạo và khởi nghiệp Công nghệ. Các lĩnh vực nghiên cứu của ThS. Hùng bao gồm Đồ họa máy tính, đa phương tiện, Thương mại điện tử, Hệ thống thông tin / ERP / MRP, Đổi mới và khởi nghiệp. Ông quan tâm đến các nghiên cứu về Chăm sóc sức khỏe, Tele-y học, Sức khỏe điện tử, Lập trình di động, và Marketing di động. Trong các 

In [10]:
raw_datasets


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 892
    })
})

In [11]:
import torch
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("vinai/bartpho-syllable")
tokenizer.is_fast

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartphoTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


True

In [13]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s> Thạc sĩ Lê Tấn Hùng hiện đang làm việc ở bộ môn nào?</s></s> Thạc sĩ Lê Tấn Hùng hiện đang làm việc tại bộ môn Công nghệ Phần mềm từ năm 2000. ThS Hùng có các hướng nghiên cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos. Email của ông là: hunglt@soict.hust.edu.vn và có trang web cá nhân tại: http://www.soict.hust.edu.vn/~hunglt. Ông đã tốt nghiệp Thạc sĩ tại Đại học Châu Á, Thái Lan năm 1998 và Cử nhân tại Đại học Tổng hợp Sofia năm 1994. ThS. Hùng giảng dạy các môn học bao gồm Đồ họa máy tính, Đa phương tiện và Hệ thống giải trí, Đồ Họa và Hiện thực ảo, Đạo đức Máy tính, và Sáng Tạo và khởi nghiệp Công nghệ. Các lĩnh vực nghiên cứu của ThS. Hùng bao gồm Đồ họa máy tính, đa phương tiện, Thương mại điện tử, Hệ thống thông tin / ERP / MRP, Đổi mới và khởi nghiệp. Ông quan tâm đến các nghiên cứu về Chăm sóc sức khỏe, Tele-y học, Sức khỏe điện tử, Lập trình di động, và Marketing di

In [14]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))
    print()

<s> Thạc sĩ Lê Tấn Hùng hiện đang làm việc ở bộ môn nào?</s></s> Thạc sĩ Lê Tấn Hùng hiện đang làm việc tại bộ môn Công nghệ Phần mềm từ năm 2000. ThS Hùng có các hướng nghiên cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos. Email của ông là: hunglt@soic</s>

<s> Thạc sĩ Lê Tấn Hùng hiện đang làm việc ở bộ môn nào?</s></s> cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos. Email của ông là: hunglt@soict.hust.edu.vn và có trang web cá nhân tại: http://www.soict.hust.edu.</s>

<s> Thạc sĩ Lê Tấn Hùng hiện đang làm việc ở bộ môn nào?</s></s>, thương mại điện tử, M pos, Pos. Email của ông là: hunglt@soict.hust.edu.vn và có trang web cá nhân tại: http://www.soict.hust.edu.vn/~hunglt. Ông đã tốt nghiệp Thạc sĩ tại Đại học Châu Á, Thái Lan năm 1998 và Cử nhân tại Đại</s>

<s> Thạc sĩ Lê T

In [15]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [16]:
inputs["overflow_to_sample_mapping"]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [17]:
inputs = tokenizer(
    raw_datasets["train"][2:5]["question"],
    raw_datasets["train"][2:5]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 3 examples gave 41 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2].


In [18]:
raw_datasets["train"][2:5]["answers"]


[{'text': ['ThS Hùng có các hướng nghiên cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos'],
  'answer_start': [82]},
 {'text': ['Email của ông là: hunglt@soict.hust.edu.vn'],
  'answer_start': [265]},
 {'text': ['trang web cá nhân tại: http://www.soict.hust.edu.vn/~hunglt'],
  'answer_start': [314]}]

In [19]:
print(raw_datasets["train"][2:5])

{'id': ['01-03', '01-04', '01-05'], 'title': ['Thạc sĩ Lê Tấn Hùng', 'Thạc sĩ Lê Tấn Hùng', 'Thạc sĩ Lê Tấn Hùng'], 'context': ['Thạc sĩ Lê Tấn Hùng hiện đang làm việc tại bộ môn Công nghệ Phần mềm từ năm 2000. ThS Hùng có các hướng nghiên cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos. Email của ông là: hunglt@soict.hust.edu.vn và có trang web cá nhân tại: http://www.soict.hust.edu.vn/~hunglt. Ông đã tốt nghiệp Thạc sĩ tại Đại học Châu Á, Thái Lan năm 1998 và Cử nhân tại Đại học Tổng hợp Sofia năm 1994. ThS. Hùng giảng dạy các môn học bao gồm Đồ họa máy tính, Đa phương tiện và Hệ thống giải trí, Đồ Họa và Hiện thực ảo, Đạo đức Máy tính, và Sáng Tạo và khởi nghiệp Công nghệ. Các lĩnh vực nghiên cứu của ThS. Hùng bao gồm Đồ họa máy tính, đa phương tiện, Thương mại điện tử, Hệ thống thông tin / ERP / MRP, Đổi mới và khởi nghiệp. Ông quan tâm đến các nghiên cứu về Chăm sóc sức khỏe, T

In [20]:
answers = raw_datasets["train"][2:5]["answers"]
print(answers)

[{'text': ['ThS Hùng có các hướng nghiên cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos'], 'answer_start': [82]}, {'text': ['Email của ông là: hunglt@soict.hust.edu.vn'], 'answer_start': [265]}, {'text': ['trang web cá nhân tại: http://www.soict.hust.edu.vn/~hunglt'], 'answer_start': [314]}]


In [21]:
answers = raw_datasets["train"][2:5]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([43,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  52,
  19,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  48,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [88,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  69,
  36,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  71,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [22]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: ThS Hùng có các hướng nghiên cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos, labels give: ThS Hùng có các hướng nghiên cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos


In [23]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: ThS Hùng có các hướng nghiên cứu, giảng dạy và triển khai thực tế: hệ thống Y tế từ xa, hệ thống marketing trên Mobile, hệ thống giải trí trên Mobile, thương mại điện tử, M pos, Pos, decoded example: <s> Các hướng nghiên cứu của Thạc sĩ Lê Tấn Hùng bao gồm những gì?</s></s>lt. Ông đã tốt nghiệp Thạc sĩ tại Đại học Châu Á, Thái Lan năm 1998 và Cử nhân tại Đại học Tổng hợp Sofia năm 1994. ThS. Hùng giảng dạy các môn học bao gồm Đồ họa máy tính, Đa phương tiện và Hệ thống giải trí, Đồ Họa và Hiện thực ảo, Đạo đức Máy tính, và Sáng Tạo và khởi nghiệp Công</s>


In [24]:
max_length = 1024
stride = 128

def preprocess_training_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        if len(answer["text"]) > 0 and answer["answer_start"][0] is not None:
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])
            sequence_ids = inputs.sequence_ids(i)

            # Find the start and end of the context
            idx = 0
            while idx < len(sequence_ids) and sequence_ids[idx] != 1:
                idx += 1
            context_start = idx

            while idx < len(sequence_ids) and sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # If the answer is not fully inside the context, label is (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Otherwise it's the start and end token positions
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)
        else:
            start_positions.append(0)
            end_positions.append(0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

train_dataset = raw_datasets["train"].map(
    preprocess_training_dataset,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)


Map:   0%|          | 0/892 [00:00<?, ? examples/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [25]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

train_dataset = raw_datasets["train"].map(
    preprocess_training_dataset,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/892 [00:00<?, ? examples/s]

(892, 892)

In [26]:
def preprocess_validation_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [27]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/892 [00:00<?, ? examples/s]

(892, 892)

In [28]:
from tqdm.auto import tqdm
import evaluate
import numpy as np
import collections

n_best = 20
max_answer_length = 30
predicted_answers = []
metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Ensure index values are within the context range
                    if (offsets[start_index] is None or offsets[end_index] is None or
                        end_index < start_index or
                        end_index - start_index + 1 > max_answer_length):
                        continue

                    # Extract the text snippet for the current predicted answer
                    start_char_pos, end_char_pos = offsets[start_index][0], offsets[end_index][1]
                    if start_char_pos is not None and end_char_pos is not None:
                        answer_text = context[start_char_pos:end_char_pos]
                        logit_score = start_logit[start_index] + end_logit[end_index]
                        answers.append({"text": answer_text, "logit_score": logit_score})

        # Select the answer with the highest logit score
        if answers:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)


In [29]:
pip install --upgrade transformers


In [30]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
checkpoint_path = "/content/drive/My Drive/checkpoint-24000"
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

In [31]:
from transformers import Trainer, TrainingArguments, AutoModelForQuestionAnswering, TrainerCallback


class PrintLossCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and 'loss' in logs:
            print(f"Epoch: {state.epoch}, Step: {state.global_step}, Loss: {logs['loss']}")

epochs = 10
batch_size = 2
lr = 2e-5

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/model2",
    evaluation_strategy="steps",  # Đánh giá dựa trên số bước
    eval_steps=100,  # Đánh giá sau mỗi 10 bước
    logging_strategy="steps",  # Ghi log sau mỗi bước
    logging_steps=100,  # Ghi log sau mỗi 10 bước
    num_train_epochs=epochs,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=1,
    save_steps=100,
    gradient_accumulation_steps=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
    callbacks=[PrintLossCallback()]
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
100,2.515200,No log
200,1.521000,No log
300,0.994500,No log
400,1.007900,No log
500,0.783200,No log
600,0.629300,No log
700,0.635400,No log
800,0.369700,No log
900,0.469000,No log
1000,0.264100,No log


Epoch: 0.4484304932735426, Step: 100, Loss: 2.5152


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


Epoch: 0.8968609865470852, Step: 200, Loss: 1.521


...

Epoch: 9.417040358744394, Step: 2100, Loss: 0.048


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


Epoch: 9.865470852017937, Step: 2200, Loss: 0.031


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


TrainOutput(global_step=2230, training_loss=0.48063327704844455, metrics={'train_runtime': 2869.3445, 'train_samples_per_second': 3.109, 'train_steps_per_second': 0.777, 'total_flos': 5439101856153600.0, 'train_loss': 0.48063327704844455, 'epoch': 10.0})

In [32]:
tokenizer.save_pretrained("/content/drive/MyDrive/model2")
model.save_pretrained("/content/drive/MyDrive/model2")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


In [35]:
import torch
def answer_question(context, question, model, tokenizer):
    # Tạo input cho model và lấy offset_mapping
    inputs = tokenizer(question, context, return_tensors="pt", add_special_tokens=True, return_offsets_mapping=True)
    offset_mapping = inputs.pop("offset_mapping")  # Lưu và loại bỏ offset_mapping khỏi inputs

    # Xóa token_type_ids nếu có
    inputs.pop("token_type_ids", None)

    # Đảm bảo model không ở chế độ training
    model.eval()

    # Dự đoán câu trả lời với torch.no_grad() để không tính toán gradient
    with torch.no_grad():
        outputs = model(**inputs)

    # Lấy câu trả lời từ output
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Tìm vị trí bắt đầu và kết thúc của câu trả lời
    answer_start = torch.argmax(answer_start_scores, dim=1).item()
    answer_end = torch.argmax(answer_end_scores, dim=1).item() + 1

    # Chuyển đổi từ chỉ số token sang chỉ số ký tự
    if answer_start >= len(offset_mapping[0]) or answer_end >= len(offset_mapping[0]):
        return "Error: Answer start or end index is out of bounds."
    else:
        start_char = offset_mapping[0][answer_start][0]
        end_char = offset_mapping[0][answer_end][1]

        # Lấy câu trả lời từ text
        answer = context[start_char:end_char]
        return answer


In [33]:
checkpoint_path = "/content/drive/MyDrive/model2"

# Tải lại mô hình và tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

In [48]:
context = """
TS. Đinh Thị Hà Ly nhận bằng Kỹ sư và Thạc sĩ chuyên ngành Khoa học máy tính tại Đại học Bách Khoa Hà Nội vào các năm 2015 và 2017, đã tiếp tục hoàn thành bằng Tiến sĩ chuyên ngành Tin học vào năm 2022 tại Viện Tin học Quốc Gia (NII), thuộc Đại học Sau Đại học dành cho Nghiên cứu Cao cấp (SOKENDAI), Nhật Bản. Hướng nghiên cứu của cô tập trung vào các thuật toán tiến hoá và các bài toán tối ưu, đặc biệt là trong lĩnh vực hệ thống truyền thông không dây và Internet vạn vật. Gần đây, cô đã tập trung vào việc áp dụng các kỹ thuật học trực tuyến để giải quyết các vấn đề liên quan đến phân bố tài nguyên hiệu quả năng lượng và kiểm soát tín hiệu nhiễu trong các mạng không dây thế hệ mới. Cô Đinh Thị Hà Ly dạy các môn học như IT3020 - Toán rời rạc, IT2120 - Kiến thức máy tính, IT3170 - Thực hành Thuật toán ứng dụng, và IT3080E - Thực hành Mạng máy tính. Địa chỉ email của cô là lydth@soict.hust.edu.vn. Trong lĩnh vực nghiên cứu, cô đã đóng góp nhiều công trình khoa học tiêu biểu, bao gồm: "Device Selection and Beamforming Optimization in Large-Scale mmWave IoT Networks" được đăng trên IEEE Internet of Things Journal vào tháng 8 năm 2022 và "Wireless Multi-Interface Connectivity with Deep Learning-Enabled User Devices: an Energy Efficiency Perspective" được đăng trên IEEE Network vào tháng 8 năm 2022 và "Improving Reliability by Risk-Averse Reinforcement Learning over Sub6GHz/mmWave Integrated Networks" được trình bày tại IEEE International Conference on Communications (ICC) vào tháng 5 năm 2022 và "Distributed User-to-Multiple Access Points Association through Deep Learning for Beyond 5G" được đăng trên Elsevier Computer Networks vào tháng 10 năm 2021 và "Deep Reinforcement Learning-based User Association in Sub6GHz/mmWave Integrated Networks" được trình bày tại IEEE 18th Annual Consumer Communications & Networking Conference (CCNC) vào tháng 1 năm 2021 và "Energy Efficient Resource Allocation Optimization in Fog Radio Access Networks with Outdated Channel Knowledge" được đăng trên IEEE Transaction on Green Communications and Networking vào tháng 10 năm 2020 và "Reinforcement Learning-aided Distributed User-to-Access Points Association in Interfering Networks" được trình bày tại IEEE Global Communications Conference (GLOBECOM) vào tháng 12 năm 2019 và "Efficient Approximation Approaches to Minimal Exposure Path Problem in Probabilistic Coverage Model for Wireless Sensor Networks" được đăng trên Applied Soft Computing vào tháng 3 năm 2019 và "Energy-Efficient User Association and Beamforming for 5G Fog Radio Access Networks" được trình bày tại IEEE 16th Annual Consumer Communications & Networking Conference (CCNC) vào tháng 1 năm 2019 và "User Pre-Scheduling and Beamforming with Outdated CSI in 5G Fog Radio Access Networks" được trình bày tại IEEE Global Communications Conference (GLOBECOM) vào tháng 12 năm 2018. Cô cũng đã được vinh danh với giải thưởng cho bài báo tiếng Anh xuất sắc nhất tại hội nghị IEICE Network Systems Conference ở Nhật Bản vào năm 2020.
"""
question = """
email của Đinh Thị Hà Ly là gì ?
"""

answer = answer_question(context, question, model, tokenizer)
print(f"Câu trả lời: {answer}")

Câu trả lời:  Địa chỉ email của cô là lydth@soict.hust.edu.vn.
